In [ ]:
# -*- coding: utf-8 -*-
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

"""
Created on Sat Sep 19 20:55:56 2015

@author: liangshiyu
"""

from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy import misc
import random

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
def odin(model=None, id_testloader=None, out_testloader=None, magnitude=0.0014, temperature=1000, std=(0,0,0), file_path=''):
    g1 = open(f"{file_path}/confidence_Our_In.txt", 'w')
    g2 = open(f"{file_path}/confidence_Our_Out.txt", 'w')
    
    print("Processing in-distribution images")
    
    criterion = nn.CrossEntropyLoss()
    
########################################In-distribution###########################################
    for j, data in enumerate(id_testloader):
        images, _ = data
        batch_size = images.size(0)
        
        inputs = images.to(device).requires_grad_()
        outputs = model(inputs)
        
        # Calculating the confidence of the output, no perturbation added here, no temperature scaling used
        nnOutputs = outputs.detach().cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        # Using temperature scaling
        outputs = outputs / temperature
	
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = torch.argmax(outputs, dim=1)
        labels = maxIndexTemp.to(device)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        
        # Normalizing the gradient to the same space of image
        gradient[0][0] = (gradient[0][0]) / std[0]
        gradient[0][1] = (gradient[0][1]) / std[1]
        gradient[0][2] = (gradient[0][2]) / std[2]
        
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.detach(), gradient, alpha=-magnitude)
        outputs = model(tempInputs)
        outputs = outputs / temperature
        
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        for i in range(batch_size):
            g1.write("{}, {}, {}\n".format(temperature, magnitude, np.max(nnOutputs[i])))
            
        images.grad = None
        
    print("Processing out-of-distribution images")
###################################Out-of-Distributions#####################################
    for j, data in enumerate(out_testloader):
        images, _ = data
        batch_size = images.size(0)
    
        inputs = images.to(device).requires_grad_()
        outputs = model(inputs)
        
        # Calculating the confidence of the output, no perturbation added here
        nnOutputs = outputs.detach().cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        # Using temperature scaling
        outputs = outputs / temperature  
  
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = torch.argmax(outputs, dim=1)
        labels = maxIndexTemp.to(device)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        
        # Normalizing the gradient to the same space of image
        gradient[0][0] = (gradient[0][0]) / std[0]
        gradient[0][1] = (gradient[0][1]) / std[1]
        gradient[0][2] = (gradient[0][2]) / std[2]
        
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.detach(), gradient, alpha=-magnitude)
        outputs = model(tempInputs)
        outputs = outputs / temperature
        
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        for i in range(batch_size):
            g2.write("{}, {}, {}\n".format(temperature, magnitude, np.max(nnOutputs[i])))
            
        images.grad = None
        
    g1.close()
    g2.close()        

In [ ]:
# import fault_injector as fi

def odin_fi(model=None, id_testloader=None, out_testloader=None, magnitude=0.0014, temperature=1000, std=(0,0,0), file_path='', 
            first_forward_fi=False, backward_fi=False, second_forward_fi=False, bit_lists=[], flip_ratio=0, layer_name='', fs=None):    
    g1 = open(f"{file_path}/confidence_Our_In_fi.txt", 'w')
    g2 = open(f"{file_path}/confidence_Our_Out_fi.txt", 'w')
    
    print("Processing in-distribution images")
    
    criterion = nn.CrossEntropyLoss()
    
########################################In-distribution###########################################
    for j, data in enumerate(id_testloader):
        images, _ = data
        batch_size = images.size(0)
        
        inputs = images.to(device).requires_grad_()     
        
        hook_handlers = []
        
        if first_forward_fi:
            hook_handler = fs.onlineMultiBitLayerOutputInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[ID] 첫번째 forward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[ID] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")
            
        outputs = model(inputs)               
        
        for handler in hook_handlers:
            handler.remove()
            
        hook_handlers = []
        
        # Calculating the confidence of the output, no perturbation added here, no temperature scaling used
        nnOutputs = outputs.detach().cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        # Using temperature scaling
        outputs = outputs / temperature
	
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = torch.argmax(outputs, dim=1)
        labels = maxIndexTemp.to(device)
        
        loss = criterion(outputs, labels)
        
        if backward_fi:
            hook_handler = fs.onlineMultiBitLayerBackwardInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[ID] Backward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[ID] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")
            
        loss.backward()
        
        for handler in hook_handlers:
            handler.remove()
        
        hook_handlers = []
                    
        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        
        # Normalizing the gradient to the same space of image
        gradient[0][0] = (gradient[0][0]) / std[0]
        gradient[0][1] = (gradient[0][1]) / std[1]
        gradient[0][2] = (gradient[0][2]) / std[2]
        
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.detach(), gradient, alpha=-magnitude)
        
        if second_forward_fi:
            hook_handler = fs.onlineMultiBitLayerOutputInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[ID] 두번째 forward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[ID] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")            
        
        outputs = model(tempInputs)      
        
        for handler in hook_handlers:
            handler.remove()       

        outputs = outputs / temperature
        
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        for i in range(batch_size):
            g1.write("{}, {}, {}\n".format(temperature, magnitude, np.max(nnOutputs[i])))
            
        images.grad = None
        
    print("Processing out-of-distribution images")
###################################Out-of-Distributions#####################################
    for j, data in enumerate(out_testloader):
        images, _ = data
        batch_size = images.size(0)
    
        inputs = images.to(device).requires_grad_()           
            
        hook_handlers = []
        
        if first_forward_fi:
            hook_handler = fs.onlineMultiBitLayerOutputInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[OOD] 첫번째 forward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[OOD] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")            
            
        outputs = model(inputs)               
        
        for handler in hook_handlers:
            handler.remove()
            
        hook_handlers = []
        
        # Calculating the confidence of the output, no perturbation added here
        nnOutputs = outputs.detach().cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        # Using temperature scaling
        outputs = outputs / temperature  
  
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = torch.argmax(outputs, dim=1)
        labels = maxIndexTemp.to(device)
        
        loss = criterion(outputs, labels)
        
        if backward_fi:
            hook_handler = fs.onlineMultiBitLayerBackwardInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[OOD] Backward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[OOD] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")            
            
        loss.backward()
        
        for handler in hook_handlers:
            handler.remove()
        
        hook_handlers = []
    
        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        
        # Normalizing the gradient to the same space of image
        gradient[0][0] = (gradient[0][0]) / std[0]
        gradient[0][1] = (gradient[0][1]) / std[1]
        gradient[0][2] = (gradient[0][2]) / std[2]
        
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.detach(), gradient, alpha=-magnitude)
        
        if second_forward_fi:
            hook_handler = fs.onlineMultiBitLayerOutputInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[OOD] 두번째 forward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[OOD] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")            
        
        outputs = model(tempInputs)      
        
        for handler in hook_handlers:
            handler.remove()    

        outputs = outputs / temperature
        
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        for i in range(batch_size):
            g2.write("{}, {}, {}\n".format(temperature, magnitude, np.max(nnOutputs[i])))
            
        images.grad = None
        
    g1.close()
    g2.close()      
    

In [ ]:
# import fault_injector as fi

def odin_fi_ash(model=None, id_testloader=None, out_testloader=None, magnitude=0.0014, temperature=1000, std=(0,0,0), file_path='', 
            first_forward_fi=False, backward_fi=False, second_forward_fi=False, bit_lists=[], flip_ratio=0, layer_name='', fs=None, ash_method=''):    
    g1 = open(f"{file_path}/confidence_Our_In_fi_ash.txt", 'w')
    g2 = open(f"{file_path}/confidence_Our_Out_fi_ash.txt", 'w')
    
    print("Processing in-distribution images")
    
    criterion = nn.CrossEntropyLoss()
    
########################################In-distribution###########################################
    for j, data in enumerate(id_testloader):
        images, _ = data
        batch_size = images.size(0)
        
        inputs = images.to(device).requires_grad_()     
        
        hook_handlers = []
        
        if first_forward_fi:
            hook_handler = fs.onlineMultiBitLayerOutputInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[ID] 첫번째 forward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[ID] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")
            
        outputs = model.forward_ash(inputs, ash_method)              
        
        for handler in hook_handlers:
            handler.remove()
            
        hook_handlers = []
        
        # Calculating the confidence of the output, no perturbation added here, no temperature scaling used
        nnOutputs = outputs.detach().cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        # Using temperature scaling
        outputs = outputs / temperature
	
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = torch.argmax(outputs, dim=1)
        labels = maxIndexTemp.to(device)
        
        loss = criterion(outputs, labels)
        
        if backward_fi:
            hook_handler = fs.onlineMultiBitLayerBackwardInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[ID] Backward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[ID] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")
            
        loss.backward()
        
        for handler in hook_handlers:
            handler.remove()
        
        hook_handlers = []
                    
        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        
        # Normalizing the gradient to the same space of image
        gradient[0][0] = (gradient[0][0]) / std[0]
        gradient[0][1] = (gradient[0][1]) / std[1]
        gradient[0][2] = (gradient[0][2]) / std[2]
        
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.detach(), gradient, alpha=-magnitude)
        
        if second_forward_fi:
            hook_handler = fs.onlineMultiBitLayerOutputInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[ID] 두번째 forward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[ID] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")            
        
        outputs = model.forward_ash(tempInputs, ash_method)       
        
        for handler in hook_handlers:
            handler.remove()       

        outputs = outputs / temperature
        
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        for i in range(batch_size):
            g1.write("{}, {}, {}\n".format(temperature, magnitude, np.max(nnOutputs[i])))
            
        images.grad = None
        
    print("Processing out-of-distribution images")
###################################Out-of-Distributions#####################################
    for j, data in enumerate(out_testloader):
        images, _ = data
        batch_size = images.size(0)
    
        inputs = images.to(device).requires_grad_()           
            
        hook_handlers = []
        
        if first_forward_fi:
            hook_handler = fs.onlineMultiBitLayerOutputInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[OOD] 첫번째 forward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[OOD] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")            
            
        outputs = model.forward_ash(inputs, ash_method)           
        
        for handler in hook_handlers:
            handler.remove()
            
        hook_handlers = []
        
        # Calculating the confidence of the output, no perturbation added here
        nnOutputs = outputs.detach().cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        # Using temperature scaling
        outputs = outputs / temperature  
  
        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        maxIndexTemp = torch.argmax(outputs, dim=1)
        labels = maxIndexTemp.to(device)
        
        loss = criterion(outputs, labels)
        
        if backward_fi:
            hook_handler = fs.onlineMultiBitLayerBackwardInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[OOD] Backward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[OOD] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")            
            
        loss.backward()
        
        for handler in hook_handlers:
            handler.remove()
        
        hook_handlers = []
    
        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(inputs.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        
        # Normalizing the gradient to the same space of image
        gradient[0][0] = (gradient[0][0]) / std[0]
        gradient[0][1] = (gradient[0][1]) / std[1]
        gradient[0][2] = (gradient[0][2]) / std[2]
        
        # Adding small perturbations to images
        tempInputs = torch.add(inputs.detach(), gradient, alpha=-magnitude)
        
        if second_forward_fi:
            hook_handler = fs.onlineMultiBitLayerOutputInjection(model=model, targetLayer=layer_name, NofError=flip_ratio, bit_positions=bit_lists)
            hook_handlers.append(hook_handler)
            # print(f"[OOD] 두번째 forward pass - 레이어 {layer_name}의 {str(bit_lists)} 비트에 {flip_ratio} 비율로 결함 주입 설정")
            
            if hasattr(fs, '_current_injection_stats'):
                stats = fs._current_injection_stats
                # print(f"[OOD] 오류 주입된 뉴런 비율: {stats['injected_neurons']}/{stats['total_neurons']} = {stats['ratio']:.4f}")            
        
        outputs = model.forward_ash(tempInputs, ash_method)       
        
        for handler in hook_handlers:
            handler.remove()    

        outputs = outputs / temperature
        
        # Calculating the confidence after adding perturbations
        nnOutputs = outputs.data.cpu().numpy()
        nnOutputs = np.exp(nnOutputs - np.max(nnOutputs, axis=1, keepdims=True))
        nnOutputs = nnOutputs / np.sum(nnOutputs, axis=1, keepdims=True)
        
        for i in range(batch_size):
            g2.write("{}, {}, {}\n".format(temperature, magnitude, np.max(nnOutputs[i])))
            
        images.grad = None
        
    g1.close()
    g2.close()      
    